In [35]:
import tensorflow as tf
import pandas

In [2]:
import numpy as np

In [3]:
data = pandas.read_csv("E:/my_courses/هوش محاسباتی/birds-bones-and-living-habits/bird.csv")

In [4]:
len(data)

420

In [5]:
train_len = test_len = cross_validation_len =140

In [6]:
batch_size = 1 #because data is under 1000
Epochs = 15

In [7]:
data.head()

,id,huml,humw,ulnal,ulnaw,feml,femw,tibl,tibw,tarl,tarw,type
0,0,80.78,6.68,72.01,4.88,41.81,3.70,5.50,4.03,38.70,3.84,SW
1,1,88.91,6.63,80.53,5.59,47.04,4.30,80.22,4.51,41.50,4.01,SW
2,2,79.97,6.37,69.26,5.28,43.07,3.90,75.35,4.04,38.31,3.34,SW
3,3,77.65,5.70,65.76,4.77,40.04,3.52,69.17,3.40,35.78,3.41,SW
4,4,62.80,4.84,52.09,3.73,33.95,2.72,56.27,2.96,31.88,3.13,SW


In [8]:
del data['id'] #remove id column because it is useless

In [9]:
data.head()

,huml,humw,ulnal,ulnaw,feml,femw,tibl,tibw,tarl,tarw,type
0,80.78,6.68,72.01,4.88,41.81,3.70,5.50,4.03,38.70,3.84,SW
1,88.91,6.63,80.53,5.59,47.04,4.30,80.22,4.51,41.50,4.01,SW
2,79.97,6.37,69.26,5.28,43.07,3.90,75.35,4.04,38.31,3.34,SW
3,77.65,5.70,65.76,4.77,40.04,3.52,69.17,3.40,35.78,3.41,SW
4,62.80,4.84,52.09,3.73,33.95,2.72,56.27,2.96,31.88,3.13,SW


In [10]:
classes = data.drop_duplicates(subset='type')['type'] #catch nomber of classes of birds

In [11]:
len(classes)

6

In [12]:
class_of_birds1 = pandas.Series(classes.values,index=np.arange(6)).to_dict()
class_of_birds2 = pandas.Series(np.arange(6),index=classes.values).to_dict()
#make dictionary of class of birds 
class_of_birds2

{'SW': 0, 'W': 1, 'T': 2, 'R': 3, 'P': 4, 'SO': 5}

In [13]:
len(data['type'])

420

In [14]:
from pandas import DataFrame
train_labels = pandas.DataFrame([class_of_birds2[data['type'][i]] for i in range(train_len)])


In [15]:
cross_validation_labels = pandas.DataFrame([class_of_birds2[data['type'][i+140]] for i in range(train_len)])


In [16]:
test_labels = pandas.DataFrame([class_of_birds2[data['type'][i+280]] for i in range(train_len)])

In [17]:
del data['type']
data.head()

,huml,humw,ulnal,ulnaw,feml,femw,tibl,tibw,tarl,tarw
0,80.78,6.68,72.01,4.88,41.81,3.70,5.50,4.03,38.70,3.84
1,88.91,6.63,80.53,5.59,47.04,4.30,80.22,4.51,41.50,4.01
2,79.97,6.37,69.26,5.28,43.07,3.90,75.35,4.04,38.31,3.34
3,77.65,5.70,65.76,4.77,40.04,3.52,69.17,3.40,35.78,3.41
4,62.80,4.84,52.09,3.73,33.95,2.72,56.27,2.96,31.88,3.13


In [18]:
#build train
train_data = data[:140] #train_data which has features in it

In [19]:
train_labels = train_labels.values

In [20]:
#build cross validation 
cross_validation_data = data[140:280]

In [21]:
cross_validation_labels = cross_validation_labels.values

In [22]:
cross_validation_data.head()

,huml,humw,ulnal,ulnaw,feml,femw,tibl,tibw,tarl,tarw
140,88.16,8.15,66.99,6.25,46.55,3.86,83.02,3.99,36.71,3.84
141,82.39,4.80,92.50,3.64,47.86,4.05,140.00,4.26,95.39,3.43
142,83.48,5.06,94.46,3.57,49.26,4.04,110.00,3.80,82.00,3.45
143,27.94,1.93,30.25,1.66,19.55,1.52,35.21,1.38,24.47,1.16
144,29.40,2.05,32.56,1.90,20.92,1.46,38.96,1.45,23.98,1.20


In [23]:
#build test
test_data = data[280:420]
test_labels = test_labels.values
test_data.head()

,huml,humw,ulnal,ulnaw,feml,femw,tibl,tibw,tarl,tarw
280,34.07,3.26,41.34,2.62,24.25,2.30,36.35,2.13,24.69,2.12
281,33.79,3.04,42.13,2.56,23.09,2.21,34.74,2.07,22.12,1.78
282,34.15,2.98,40.67,2.45,24.72,1.98,37.35,1.93,23.49,1.67
283,36.88,3.18,46.28,2.77,26.08,2.19,39.44,2.03,25.60,1.98
284,28.55,2.65,33.74,2.38,19.56,1.98,29.21,1.64,18.75,1.53


In [24]:
#Use tf.data.Dataset.from_tensor_slices to read the values from a pandas dataframe. 
tensor_dataset = tf.data.Dataset.from_tensor_slices((train_data.values, train_labels))

In [25]:
tensor_cross_dataset = tf.data.Dataset.from_tensor_slices((cross_validation_data.values, cross_validation_labels))

In [26]:
tensor_cross_dataset = tensor_cross_dataset.shuffle(140).batch(1)

In [27]:
tensor_test_dataset = tf.data.Dataset.from_tensor_slices((test_data.values, test_labels))

In [51]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(6),
    tf.keras.layers.Softmax()
])


In [56]:
len(model.weights)

4

In [52]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [53]:
tensor_dataset = tensor_dataset.shuffle(140).batch(1)

In [54]:
model.fit(tensor_dataset, epochs=Epochs)

Epoch 1/15
140/140 [==============================] - 0s 2ms/step - loss: 1.8722 - acc: 0.1714
Epoch 2/15
140/140 [==============================] - 0s 939us/step - loss: 1.8722 - acc: 0.1714
Epoch 3/15
140/140 [==============================] - 0s 986us/step - loss: 1.8722 - acc: 0.1714
Epoch 4/15
140/140 [==============================] - 0s 1ms/step - loss: 1.8846 - acc: 0.1571
Epoch 5/15
140/140 [==============================] - 0s 1ms/step - loss: 1.8722 - acc: 0.1714
Epoch 6/15
140/140 [==============================] - 0s 1ms/step - loss: 1.8722 - acc: 0.1714
Epoch 7/15
140/140 [==============================] - 0s 986us/step - loss: 1.8722 - acc: 0.1714
Epoch 8/15
140/140 [==============================] - 0s 828us/step - loss: 1.8722 - acc: 0.1714
Epoch 9/15
140/140 [==============================] - 0s 939us/step - loss: 1.8722 - acc: 0.1714
Epoch 10/15
140/140 [==============================] - 0s 986us/step - loss: 1.8722 - acc: 0.1714
Epoch 11/15
140/140 [================

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  110       
_________________________________________________________________
dense_5 (Dense)              multiple                  66        
_________________________________________________________________
softmax_2 (Softmax)          multiple                  0         
Total params: 176
Trainable params: 176
Non-trainable params: 0
_________________________________________________________________


In [55]:
cross_loss, cross_acc = model.evaluate(tensor_dataset, verbose=2)



140/140 - 0s - loss: 1.2150 - acc: 0.8286
